In [3]:
!pip install transformers

     |████████████████████████████████| 1.9MB 17.6MB/s 
     |████████████████████████████████| 3.2MB 42.6MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=096166cbe3919f12a41251a4f45a7d4f220e40f387059e91141416abe9c77f17
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
consumo_dir = '/content/drive/MyDrive/0AccTeam/datasets_consumo/'

In [8]:
data = pd.read_csv(consumo_dir +"data_training.csv")
data_test = pd.read_csv(consumo_dir +"data_testing.csv")

In [9]:
MODEL_NAME = 'bert-base-uncased'

In [10]:
MAX_LENGTH = 100

In [11]:
# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(MODEL_NAME)
config.output_hidden_states = False

In [12]:
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = MODEL_NAME, config = config)

In [13]:
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(MODEL_NAME, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [14]:
# Load the MainLayer
bert = transformer_model.layers[0]

In [15]:
# Build your model input
input_ids = Input(shape=(MAX_LENGTH,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

In [16]:
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

In [17]:
# Then build your model output
food = Dense(units=len(data.food_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='food')(pooled_output)
service = Dense(units=len(data.service_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='service')(pooled_output)
outputs = {'food': food, 'service': service}

In [18]:
# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

In [19]:
# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
__________________________________________________________________________________________________
pooled_output (Dropout)         (None, 768)          0           bert[0][1]                       
__________________________________________________________________________________________________
food (Dense)                    (None, 2)            1538        pooled_output[0][0]              
_________________________________________________________________________

In [20]:
#######################################
### ------- Train the model ------- ###

In [21]:
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

In [22]:
# Set loss and metrics
loss = {'food': CategoricalCrossentropy(from_logits = True), 'service': CategoricalCrossentropy(from_logits = True)}
metric = {'food': CategoricalAccuracy('recall'), 'service': CategoricalAccuracy('recall')}

In [23]:
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [24]:
# Ready output data for the model
y_food = to_categorical(data['food_label'])
y_service = to_categorical(data['service_label'])

In [25]:
# Tokenize the input (takes some time)
x = tokenizer(
    text=data['texto'].to_list(),
    add_special_tokens=True,
    max_length=MAX_LENGTH,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [26]:
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'food': y_food, 'service': y_service},
    validation_split=0.2,
    batch_size=64,
    epochs=10)

Epoch 1/10
990/990 [==============================] - 1363s 1s/step - loss: 0.7315 - food_loss: 0.3741 - service_loss: 0.3574 - food_recall: 0.8070 - service_recall: 0.8263 - val_loss: 0.3970 - val_food_loss: 0.2021 - val_service_loss: 0.1949 - val_food_recall: 0.9138 - val_service_recall: 0.9151
Epoch 2/10
990/990 [==============================] - 1353s 1s/step - loss: 0.3464 - food_loss: 0.1711 - service_loss: 0.1753 - food_recall: 0.9265 - service_recall: 0.9257 - val_loss: 0.3810 - val_food_loss: 0.1950 - val_service_loss: 0.1861 - val_food_recall: 0.9153 - val_service_recall: 0.9211
Epoch 3/10
990/990 [==============================] - 1356s 1s/step - loss: 0.3024 - food_loss: 0.1493 - service_loss: 0.1530 - food_recall: 0.9366 - service_recall: 0.9356 - val_loss: 0.3825 - val_food_loss: 0.2113 - val_service_loss: 0.1712 - val_food_recall: 0.9116 - val_service_recall: 0.9294
Epoch 4/10
990/990 [==============================] - 1354s 1s/step - loss: 0.2786 - food_loss: 0.1363 - s

In [27]:
model.save('/content/drive/MyDrive/0AccTeam/finalModel.h5')

In [28]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/0AccTeam/finalModel.h5')

In [29]:
# Ready test data
test_y_food = to_categorical(data_test['food_label'])
test_y_service = to_categorical(data_test['service_label'])
test_x = tokenizer(
    text=data_test['texto'].to_list(),
    add_special_tokens=True,
    max_length=MAX_LENGTH,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
# Run evaluation
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'food': test_y_food, 'service': test_y_service}
)

619/619 [==============================] - 131s 212ms/step - loss: 0.3795 - food_loss: 0.2045 - service_loss: 0.1750 - food_recall: 0.9181 - service_recall: 0.9318
